In [0]:
%pip install xmltodict # this is usually done in the compute instance settings under "Library"

# Pubmed Data Ingestion

This notebook handles the ingestion of raw data from the Pubmed FTP server csv file(s)

In [0]:
import dlt
import xmltodict
import json
import gzip
import tempfile
from ftplib import FTP
import logging
from pyspark.sql.types import StructType, StructField, StringType,ArrayType
import pyspark.sql.functions as F
from pyspark.sql.functions import udf

# Setup Python logger
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


In [0]:
# FTP file functions
def pubmed_ftp_client():
    ftp = FTP('ftp.ncbi.nlm.nih.gov')
    ftp.login()
    ftp.cwd('/pubmed/updatefiles/')
    return ftp

def retrieve_file(ftp_client, filename):
    local_filename = tempfile.mkstemp()[1]

    with open(local_filename, 'wb') as f:
        ftp_client.retrbinary(f'RETR {filename}', f.write)

    logger.info(f'Retrieved {filename} as {local_filename}')
    return local_filename

In [0]:
def get_xml_content(xml_gz_filename):
    with gzip.open(xml_gz_filename, "rb") as xml_file:
        return xml_file.read()


# Define table for raw Pubmed data (converted to JSON)

In [0]:
## Justin suggestions
@dlt.table(
    comment="PubMed data converted to JSON",
    table_properties={"quality": "bronze"}
)
def pubmed_ingestion():
    ftp = pubmed_ftp_client()
    remote_filenames = sorted([f for f in ftp.nlst() if f.endswith('.xml.gz')])[:10]
    ftp.quit()

    # create dataframe using these filenames
    remote_files = spark.createDataFrame([(filename,) for filename in remote_filenames], ["pubmed_fname"]).repartition(8)

    @udf(ArrayType(StringType()))
    def get_pubmed_json(fname):
        ftp = pubmed_ftp_client()
        local_fname = retrieve_file(ftp, fname)
        ftp.quit()
        xml_content = get_xml_content(local_fname)
        xml_dict = xmltodict.parse(xml_content)

        # I am unsure about this part but if you put all of these items into a list, you can then explode the DF (so I am putting something here, just unsure if it will actually work)
        return [json.dumps(item) for item in xml_dict['PubmedArticleSet']['PubmedArticle']]

    pubmed_df = remote_files.withColumn("json_content", get_pubmed_json(F.col("pubmed_fname"))) \
        .select('pubmed_fname', F.explode('json_content').alias('json_content_row')) \
        .repartition(8) \
        .withColumn("ingestion_timestamp", F.current_timestamp())

    pubmed_processed_files = pubmed_df.cache().count()

    logger.info(f"PubMed data ingestion complete. Total files processed: {pubmed_processed_files}")
    return pubmed_df